In [ ]:
#!pip install --user git+https://github.com/ultralytics/ultralytics.git@main
!pip install --user --upgrade ultralytics

In [ ]:
!pip install -U ray[tune]

In [ ]:
import sys
sys.path.append('/users/vesalaia/.local/lib/python3.9/site-packages/lib/python3.9/site-packages')
%matplotlib inline
import matplotlib.pyplot as plt
from ultralytics import YOLO
import os

In [ ]:
model = YOLO("yolov8x-seg.pt")

In [ ]:

results = model.train(
        batch=4,
        device="cuda",
        data="/scratch/project_2005488/Muutto/muutto-reg.yaml",
        epochs=100,
        imgsz=640
    )

In [ ]:
best_saved_model = "/users/vesalaia/1 YOLOv8/runs/segment/train3/weights/best.pt" 
model = YOLO(best_saved_model)

In [ ]:
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
device = "cuda"
def predict_image(model, imagepath, yolo=False):
    
    image = cv2.imread(imagepath)
    height, width, channels = image.shape

    if not yolo: # Unet, Unet++, Mask R-CNN
        mean=[0.485, 0.456, 0.406]
        std=[0.229, 0.224, 0.225]    
        model.eval()
        image = convert_from_cv2_to_image(image)
        t = T.Compose([T.Resize((1024,1024)), T.ToTensor(), T.Normalize(mean, std)])
        image = t(image)
        model.to(device);
        image=image.to(device)
        with torch.no_grad():  
            image = image.unsqueeze(0)
            output = model(image)
    else:
        imgsz = (640,640)
        resized_img = cv2.resize(image, imgsz)
        results = model.predict(resized_img)
        output = results[0]
    return output, height, width

In [ ]:
imgsz =(640,640)
#image_path = '/scratch/project_2005488/Henkikirjataulukot/Tiedosto 100.jpg'
#image_path = "/scratch/project_2005488/McGill/1778_history-england-invasion-caesar_DA30H91778_v1_original/rbsc_hume_history-england-invasion-caesar_DA30H91778_v1-0101.jpg"
test_dir = "/scratch/project_2005488/Muutto/Test"
files = [x for x in os.listdir(test_dir) if x.endswith(".jpg")]
for i, filename in enumerate(files):
    image_path = os.path.join(test_dir, filename)

    img = cv2.imread(image_path)
    result, o_rows, o_cols = predict_image(model, image_path, yolo=True)
    for box in result.boxes:
        if box.conf[0].item() >= 0.4:
            class_id = result.names[box.cls[0].item()]
            coords = box.xyxy[0].tolist()
            coords = [coords[0] * o_cols/imgsz[0], 
                      coords[1] * o_rows/imgsz[1], 
                      coords[2] * o_cols / imgsz[0],
                      coords[3] * o_rows /imgsz[1]]
            coords = [round(x) for x in coords]
            start_point = (coords[0],coords[1])
            end_point = (coords[2],coords[3])
            cv2.rectangle(img, start_point, end_point, color=(255,0,0),thickness=2)
            conf = round(box.conf[0].item(), 2)
            #print("Object type:", class_id)
            #print("Coordinates:", coords)
            #print("Probability:", conf)
            #print("---")
    plt.figure(figsize=(10, 16))
    cv2.imwrite("Yolo_regions_{}.jpg".format(i),img) 
    plt.imshow(img)
    plt.show()